By deploying or using this software you agree to comply with the [AI Hub Terms of Service]( https://aihub.cloud.google.com/u/0/aihub-tos) and the [Google APIs Terms of Service](https://developers.google.com/terms/). To the extent of a direct conflict of terms, the AI Hub Terms of Service will control.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/GoogleCloudPlatform/ai-platform-samples/blob/main/notebooks/samples/aihub/knn_classification/knn_classification.ipynb)

# Overview

This notebook provides an example workflow of using the [K-Nearest Neighbors ML container](https://aihub.cloud.google.com/u/0/p/products%2F9d576c4f-e774-4626-b19e-ff5f9dd2d7e6) for training a classification ML model.

### Dataset

The notebook uses the [Iris dataset](https://en.wikipedia.org/wiki/Iris_flower_data_set). It consists of 3 different types of irises  (Setosa, Versicolour, and Virginica) petal and sepal length, stored in a 150x5 table. The target variable is preprocessed to integer IDs starting from zero.

### Objective

The goal of this notebook is to go through a common training workflow:
- Create a dataset
- Train an ML model using the [AI Platform Training](https://cloud.google.com/ai-platform/training/docs) service
- Identify if the model was trained successfully by looking at the generated "Run Report"

### Costs 

This tutorial uses billable components of Google Cloud Platform (GCP):

* Cloud AI Platform
* Cloud Storage

Learn about [Cloud AI Platform
pricing](https://cloud.google.com/ml-engine/docs/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Set up your local development environment

**If you are using Colab or AI Platform Notebooks**, your environment already meets
all the requirements to run this notebook. You can skip this step.

**Otherwise**, make sure your environment meets this notebook's requirements.
You need the following:

* The Google Cloud SDK
* Git
* Python 3
* virtualenv
* Jupyter notebook running in a virtual environment with Python 3

The Google Cloud guide to [Setting up a Python development
environment](https://cloud.google.com/python/setup) and the [Jupyter
installation guide](https://jupyter.org/install) provide detailed instructions
for meeting these requirements. The following steps provide a condensed set of
instructions:

1. [Install and initialize the Cloud SDK.](https://cloud.google.com/sdk/docs/)

2. [Install Python 3.](https://cloud.google.com/python/setup#installing_python)

3. [Install
   virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)
   and create a virtual environment that uses Python 3.

4. Activate that environment and run `pip install jupyter` in a shell to install
   Jupyter.

5. Run `jupyter notebook` in a shell to launch Jupyter.

6. Open this notebook in the Jupyter Notebook Dashboard.

### Set up your GCP project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a GCP project.](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project.](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [Enable the AI Platform APIs and Compute Engine APIs.](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. Enter your project ID in the cell below. Then run the  cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

In [ ]:
PROJECT_ID = "[your-project-id]" #@param {type:"string"}
! gcloud config set project $PROJECT_ID

### Authenticate your GCP account

**If you are using AI Platform Notebooks**, your environment is already
authenticated. Skip this step.

**If you are using Colab**, run the cell below and follow the instructions
when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

1. In the GCP Console, go to the [**Create service account key**
   page](https://console.cloud.google.com/apis/credentials/serviceaccountkey).

2. From the **Service account** drop-down list, select **New service account**.

3. In the **Service account name** field, enter a name.

4. From the **Role** drop-down list, select
   **Machine Learning Engine > AI Platform Admin** and
   **Storage > Storage Object Admin**.

5. Click *Create*. A JSON file that contains your key downloads to your
local environment.

6. Enter the path to your service account key as the
`GOOGLE_APPLICATION_CREDENTIALS` variable in the cell below and run the cell.

In [ ]:
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

if 'google.colab' in sys.modules:
  from google.colab import auth as google_auth
  google_auth.authenticate_user()

# If you are running this notebook locally, replace the string below with the
# path to your service account key and run this cell to authenticate your GCP
# account.
else:
  %env GOOGLE_APPLICATION_CREDENTIALS ''

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

You need to have a "workspace" bucket that will hold the dataset and the output from the ML Container. Set the name of your Cloud Storage bucket below. It must be unique across all Cloud Storage buckets. 

You may also change the `REGION` variable, which is used for operations
throughout the rest of this notebook. Make sure to [choose a region where Cloud AI Platform services are available](https://cloud.google.com/ml-engine/docs/tensorflow/regions). You may not use a Multi-Regional Storage bucket for training with AI Platform.

In [ ]:
BUCKET_NAME = "[your-bucket-name]" #@param {type:"string"}
REGION = 'us-central1' #@param {type:"string"}

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION gs://$BUCKET_NAME

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
! gsutil ls -al gs://$BUCKET_NAME

### Import libraries and define constants

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from sklearn import datasets
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import os
import time
from IPython.core.display import HTML

## Create a dataset

In [4]:
# load Iris dataset
iris = datasets.load_iris()
names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
data = pd.DataFrame(iris.data, columns=names)

# add target
data['target'] = iris.target

# split
training, validation = train_test_split(data, test_size=50, stratify=data['target'])

# standardization
training_targets = training.pop('target')
validation_targets = validation.pop('target')

data_mean = training.mean(axis=0)
data_std = training.std(axis=0)
training = (training - data_mean) / data_std
training['target'] = training_targets

validation = (validation - data_mean) / data_std
validation['target'] = validation_targets

print('Training data head')
display(training.head())

training_data = os.path.join('gs://', BUCKET_NAME, 'data/train.csv')
validation_data = os.path.join('gs://', BUCKET_NAME, 'data/valid.csv')

print('Copy the data in bucket ...')
with tf.io.gfile.GFile(training_data, 'w') as f:
  training.to_csv(f, index=False)
with tf.io.gfile.GFile(validation_data, 'w') as f:
  validation.to_csv(f, index=False)

Training data head


,sepal_length,sepal_width,petal_length,petal_width,target
87,0.579039,-1.684739,0.373408,0.150740,1
4,-0.985932,1.274938,-1.311138,-1.316643,0
96,-0.143255,-0.318734,0.261105,0.150740,1
84,-0.504402,-0.091067,0.429559,0.417537,1
120,1.301334,0.364268,1.103377,1.484725,2


Copy the data in bucket ...


## Cloud training

### Accelerator and distribution support

| GPU | Multi-GPU Node | TPU | Workers | Parameter Server |
|---|---|---|---|---|
| Yes | Yes | No | No | No |

To have distribution and/or accelerators to your AI Platform training call, use parameters similar to the examples as shown below.
```bash
    --master-machine-type standard_gpu
```
### AI Platform training

- [KNN ML container documentation](https://aihub.cloud.google.com/u/0/p/products%2F9d576c4f-e774-4626-b19e-ff5f9dd2d7e6).
- [AI Platform training documentation](https://cloud.google.com/sdk/gcloud/reference/ai-platform/jobs/submit/training).

## Local Training

In [ ]:
output_location = os.path.join('gs://', BUCKET_NAME, 'output')

job_name = "knn_classification_{}".format(time.strftime("%Y%m%d%H%M%S"))
!gcloud ai-platform jobs submit training $job_name \
    --master-image-uri gcr.io/aihub-c2t-containers/kfp-components/oob_algorithm/knn:latest \
    --region $REGION \
    --scale-tier CUSTOM \
    --master-machine-type standard \
    -- \
    --output-location {output_location} \
    --training-data {training_data} \
    --validation-data {validation_data} \
    --target-column target \
    --data-type csv \
    --k-neighbors 5 \
    --fresh-start True \
    --objective classification

### Local training snippet

Note that the training can also be done locally with Docker
```bash
docker run \
    -v /tmp:/tmp \
    -it gcr.io/aihub-c2t-containers/kfp-components/oob_algorithm/knn:latest \
    --output-location /tmp/knn_classification \
    --training-data /tmp/iris_train.csv \
    --validation-data /tmp/iris_valid.csv \
    --target-column target \
    --data-type csv \
    --k-neighbors 5 \
    --objective classification
```

## Inspect the Run Report

The "Run Report" will help you identify if the model was successfully trained.

In [6]:
if not tf.io.gfile.exists(os.path.join(output_location, 'report.html')):
  raise RuntimeError('The file report.html was not found. Did the training job finish?')

with tf.io.gfile.GFile(os.path.join(output_location, 'report.html')) as f:
  display(HTML(f.read()))

# Cleaning up

To clean up all GCP resources used in this project, you can [delete the GCP
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

In [ ]:
# If training job is still running, cancel it
! gcloud ai-platform jobs cancel $job_name --quiet

# Delete Cloud Storage objects that were created
! gsutil -m rm -r $BUCKET_NAME